في الاول هنعمل كريت ل الداتا ونخزنها في مونجو دي بي 

In [ ]:
from pymongo import MongoClient
import random

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"]

# ----- Prepare first and last names -----
first_names = ["Ahmed", "Ali", "Sara", "Mona", "Omar", "Youssef", "Nour", "Laila", "Khaled", "Hana"]
last_names = ["Refat", "Hassan", "Ibrahim", "Mohamed", "Farouk", "Adel", "Sami", "Nabil", "Tarek", "Fathy"]

# ----- Generate Users -----
users = []
for i in range(1, 21):  # 20 users
    user = {
        "user_id": i,
        "name": f"{random.choice(first_names)} {random.choice(last_names)}",
        "email": f"user{i}@example.com",
        "age": random.randint(18, 60)
    }
    users.append(user)

# Insert users
db["users"].insert_many(users)
print("Inserted 20 users")

# ----- Generate Products -----
products = []
categories = ["Electronics", "Books", "Clothes", "Sports"]
for i in range(1, 21):  # 20 products
    product = {
        "product_id": i,
        "name": f"Product {i}",
        "category": random.choice(categories),
        "price": round(random.uniform(10, 500), 2)
    }
    products.append(product)

# Insert products
db["products"].insert_many(products)
print("Inserted 20 products")







reviews = []
for i in range(1, 21):  # 20 reviews
    review = {
        "review_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "rating": random.randint(1, 5),
        "comment": f"This is a comment for product {random.randint(1, 20)}"
    }
    reviews.append(review)

# Insert reviews
db["reviews"].insert_many(reviews)
print("Inserted 20 reviews")


orders = []
for i in range(1, 21):  # 20 orders
    order = {
        "order_id": i,
        "user_id": random.randint(1, 20),  # user_id موجود
        "products": [
            {"product_id": random.randint(1, 20), "quantity": random.randint(1, 5)}
            for _ in range(random.randint(1, 3))  # كل order فيها 1-3 منتجات
        ],
        "total": 0  # هنحسبها بعدين
    }
    # حساب total
    order["total"] = sum([p["quantity"] * round(random.uniform(10, 500), 2) for p in order["products"]])
    orders.append(order)

# Insert orders
db["orders"].insert_many(orders)
print("Inserted 20 orders")
orders = []




Inserted 20 users
Inserted 20 products


In [ ]:
#  create csv files 
import pandas as pd
import random

# table 1
users_sql = pd.DataFrame({
    "user_id": range(1, 21),
    "name": [f"User_{i}" for i in range(1, 21)],
    "email": [f"user{i}@sql.com" for i in range(1, 21)],
    "signup_date": pd.date_range("2025-01-01", periods=20)
})
users_sql.to_csv("users_sql.csv", index=False)


# table 2
products_sql = pd.DataFrame({
    "product_id": range(1, 21),
    "name": [f"Product_{i}" for i in range(1, 21)],
    "category": [random.choice(["Electronics", "Books", "Clothing"]) for _ in range(20)],
    "price": [round(random.uniform(10, 500), 2) for _ in range(20)]
})
products_sql.to_csv("products_sql.csv", index=False)

# table 3

orders_sql = []
for i in range(1, 21):
    orders_sql.append({
        "order_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "quantity": random.randint(1, 5)
    })

orders_sql_df = pd.DataFrame(orders_sql)
orders_sql_df.to_csv("orders_sql.csv", index=False)


# table 4

reviews_sql = []
for i in range(1, 21):
    reviews_sql.append({
        "review_id": i,
        "user_id": random.randint(1, 20),
        "product_id": random.randint(1, 20),
        "rating": random.randint(1, 5),
        "comment": f"This is SQL comment for product {random.randint(1, 20)}"
    })

reviews_sql_df = pd.DataFrame(reviews_sql)
reviews_sql_df.to_csv("reviews_sql.csv", index=False)


intrgation of data and the processing 
---------------------

In [9]:
# اول خطوه قراءه البيانات من mongodb

from pymongo import MongoClient
import pandas as pd
from pyspark.sql   import SparkSession

# create  sparksession 
spark = SparkSession.builder.appName("firt_ETL").getOrCreate()


# the connection of mongodb local 
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"] # دي اسم الداتا بيز بتاعتي في مونجو دي بي 

collection1= pd.DataFrame(list(db["users"].find())).drop(columns=["_id"])
collection2= pd.DataFrame(list(db["orders"].find())).drop(columns=["_id"])
collection3= pd.DataFrame(list(db["products"].find())).drop(columns=["_id"])




# collection قراءه ال 
df_users =spark.createDataFrame(collection1)
df_orders =spark.createDataFrame(collection2)
df_products =spark.createDataFrame(collection3)


df_users.show(5)
df_orders.show(5)
df_products.show(5)

ConnectionRefusedError: [Errno 111] Connection refused

In [7]:
# هنا بقا هنعمل عمليه تحويل الملفات csv  ل داتا فريم جوا باي سبارك 

# قراءة ملفات CSV في PySpark
df_users_csv = spark.read.csv("users_sql.csv", header=True, inferSchema=True)
df_products_csv = spark.read.csv("products_sql.csv", header=True, inferSchema=True)
df_orders_csv = spark.read.csv("orders_sql.csv", header=True, inferSchema=True)
df_reviews_csv = spark.read.csv("reviews_sql.csv", header=True, inferSchema=True)

# عرض أول 5 صفوف لأي ملف للتأكد
df_users_csv.show(5)
#df_products_csv.show(5)
df_users_csv.printSchema()

+-------+------+-------------+-----------+
|user_id|  name|        email|signup_date|
+-------+------+-------------+-----------+
|      1|User_1|user1@sql.com| 2025-01-01|
|      2|User_2|user2@sql.com| 2025-01-02|
|      3|User_3|user3@sql.com| 2025-01-03|
|      4|User_4|user4@sql.com| 2025-01-04|
|      5|User_5|user5@sql.com| 2025-01-05|
+-------+------+-------------+-----------+
only showing top 5 rows
root
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- signup_date: date (nullable = true)

